### Generate ensembles from equilibrated state

From the initial conditions, add randomness and run model forward using 10 ensemble members. When do members diverge and become independent?



In [1]:
import numpy as np
import xarray as xr
import pyqg
from pyqg import diagnostic_tools as tools
from pyqg import qg_model, particles
import matplotlib.pyplot as plt
from numpy.random import Generator, MT19937, SeedSequence, seed
import os
import yaml

<frozen importlib._bootstrap>:228: UserWarning: No pyfftw detected. Using numpy.fft


#### Import initial conditions from equilibrium run

In [2]:
ds_initial = xr.open_dataset('/burg/abernathey/users/hillary/QG_equilibrium_proto.nc')


In [15]:
10*31104000

311040000

#### Ensemble Generator

In [ ]:
# config
year = 24*60*60*360.
day = 24*60*60.
tmax = 2*year
twrite = day
tavestart = day
sig = 1.e-6
n=1

config={'tmax':(2*year), 'twrite': (day), 'tavestart':(day), 'sig':(1.e-6)}

def ensemble_generator(initial_PV, n):
    os.mkdir('/burg/abernathey/users/hillary/'+ str('%03d'%int(n)))
    
    # configure model
    m = pyqg.QGModel(tmax=config['tmax'], twrite=config['twrite'], tavestart=config['tavestart'])
    m = pyqg.QGModel(tmax=config['tmax'], twrite=config['twrite'], tavestart=config['tavestart'])

    # create an empty array filled with zeros 
    noise = np.zeros_like(ds_initial.q.values)

    # index to the middle of noise and add random perturbation to both levels
    rg = Generator(MT19937(int(n))) 
    noise[:,np.floor(len(ds_initial.x)/2).astype('int'), 
          np.floor(len(ds_initial.y)/2).astype('int')] = config['sig']*rg.random((m.q.shape[0]))

    # set PV anomaly with randomness
    m.set_q(ds_initial.q.values + noise)
    
    # run with snapshots, save model increments as xarray DataSet
    for snapshot in m.run_with_snapshots(tsnapstart=m.t, tsnapint=m.dt):
        model = m.to_dataset()
        model = model.expand_dims(dim='n')
        model['n'] = [int(n)]
        fn = '/burg/abernathey/users/hillary/'+ str('%03d'%int(n)) +'/QG_proto_EM_'+ str('%03d_%d'%(int(n),model.time.values[0])) +'.nc'        
        model.to_netcdf(fn, engine='h5netcdf', invalid_netcdf=True, mode='a')



In [ ]:
m = ensemble_generator(ds_initial, 1)

In [ ]:
fn = '/burg/abernathey/users/hillary/'+ str('%03d'%int(n)) +'/QG_proto_EM_'+ str('%03d_%09d'%(int(n),model.time.values[0])) +'.nc'        


In [ ]:
fn